# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    
    sentences_source = source_text.split('\n')
    sentences_target = target_text.split('\n')
    
    source_id_text = []
    target_id_text = []
  
    for j in range(0,len(sentences_source)):
        source_id_text_aux = []
        source_words = sentences_source[j].split()
        for i in range(0,len(source_words)):
            source_id_text_aux.append(source_vocab_to_int[source_words[i]])
        source_id_text.append(source_id_text_aux)
    
    for j in range(0,len(sentences_target)):
        target_id_text_aux = []
        target_words = sentences_target[j].split()
        for i in range(0,len(target_words)):
            target_id_text_aux.append(target_vocab_to_int[target_words[i]])
            if i == len(target_words)-1:
                target_id_text_aux.append(target_vocab_to_int['<EOS>'])
        target_id_text.append(target_id_text_aux)
    
 
    return source_id_text, target_id_text

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, shape=(None, None), name="input")
    targets = tf.placeholder(tf.int32, shape=(None, None), name="targets")
    learning_rate = tf.placeholder(tf.float32, shape=None, name="learning_rate")
    keep_prob = tf.placeholder(tf.float32, shape=(), name="keep_prob")
    
    target_sequence_length = tf.placeholder(tf.int32, shape=(None,), name="target_sequence_length")
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name="max_target_len")
    source_sequence_length = tf.placeholder(tf.int32, shape=(None,), name="source_sequence_length")
    
    return input,targets,learning_rate,keep_prob,target_sequence_length,max_target_sequence_length,source_sequence_length


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'assert_rank_2/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "C:\\Users\\vrsilva\\AppData\\Local\\Continuum\\anaconda3\\envs\\tensorflow\\lib\\runpy.py", line 184, in _run_module_as_main\n    "__main__", mod_spec)', 'File "C:\\Users\\vrsilva\\AppData\\Local\\Continuum\\anaconda3\\envs\\tensorflow\\lib\\runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "C:\\Users\\vrsilva\\AppData\\Local\\Continuum\\anaconda3\\envs\\tensorflow\\lib\\site-packages\\ipykernel_launcher.py", line 16, in <module>\n    app.launch_new_instance()', 'File "C:\\Users\\vrsilva\\AppData\\Local\\Continuum\\anaconda3\\envs\\tensorflow\\lib\\site-packages\\traitlets\\config\\application.py", line 658, in launch_instance\n    app.start()', 'File "C:\\Users\\vrsilva\\AppDat

### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    
    remove_end = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])

    GO_ID = []
    for i in range(0,batch_size):
        GO_ID.append([target_vocab_to_int['<GO>']])

    prep_target_data = tf.concat([GO_ID,remove_end],1)
    
    return prep_target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    embed_input = tf.contrib.layers.embed_sequence(rnn_inputs,
                                                   vocab_size=source_vocab_size,
                                                   embed_dim=encoding_embedding_size,)
    def lstm_cell():
        lstm = tf.contrib.rnn.LSTMCell(num_units=rnn_size,state_is_tuple=True)
        lstm = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm
    
    
    lstm_cell_stack = tf.contrib.rnn.MultiRNNCell(cells=[lstm_cell() for _ in range(num_layers)],state_is_tuple=True)
    
    
    rnn_outputs, final_state = tf.nn.dynamic_rnn(cell=lstm_cell_stack,
                                                 inputs=embed_input,
                                                 sequence_length=source_sequence_length,
                                                 dtype=tf.float32)
                               
    
    return rnn_outputs, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    
    training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                        sequence_length=target_sequence_length,
                                                        time_major=False)
    
    training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                       training_helper,
                                                       encoder_state,
                                                       output_layer)
    
    BasicDecoderOutput = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                              impute_finished=True,
                                                              maximum_iterations=max_summary_length)[0]
    
    return BasicDecoderOutput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
   
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_tokens')
    
    inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                                start_tokens,
                                                                end_of_sequence_id)

       
    inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                        inference_helper,
                                                        encoder_state,
                                                        output_layer)
        
        
    BasicDecoderOutput = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)[0]
    
    
    return BasicDecoderOutput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # 1. Decoder Embedding
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    with tf.variable_scope("decode"):
        Training_BasicDecoderOutput = decoding_layer_train(encoder_state, dec_cell, dec_embed_input,
                                                           target_sequence_length, max_target_sequence_length,
                                                           output_layer, keep_prob)
    
    with tf.variable_scope("decode", reuse=True):
        Inference_BasicDecoderOutput = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings,
                                                            target_vocab_to_int['<GO>'], target_vocab_to_int['<EOS>'],
                                                            max_target_sequence_length, target_vocab_size, output_layer,
                                                            batch_size, keep_prob)
    
    return Training_BasicDecoderOutput, Inference_BasicDecoderOutput



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    # Pass the input data through the encoder. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data,
                                  rnn_size,
                                  num_layers,
                                  keep_prob, 
                                  source_sequence_length,
                                  source_vocab_size, 
                                  enc_embedding_size)

    
    
    # Prepare the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
               
    # Pass encoder state and decoder inputs to the decoders
    Training_BasicDecoderOutput, Inference_BasicDecoderOutput = decoding_layer(dec_input,
                                                                       enc_state,
                                                                       target_sequence_length,
                                                                       max_target_sentence_length,
                                                                       rnn_size,
                                                                       num_layers,
                                                                       target_vocab_to_int,
                                                                       target_vocab_size,
                                                                       batch_size,
                                                                       keep_prob,
                                                                       dec_embedding_size)
                                                        
                                                        
    
    return Training_BasicDecoderOutput, Inference_BasicDecoderOutput


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [5]:
# Number of Epochs
epochs = 100
# Batch Size
batch_size = 1028
# RNN Size
rnn_size = 512
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 256
decoding_embedding_size = 256
# Learning Rate
learning_rate = 0.001
# Dropout Keep Probability
keep_probability = 0.5
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [27]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [28]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [29]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/134 - Train Accuracy: 0.2386, Validation Accuracy: 0.3101, Loss: 4.7964
Epoch   0 Batch    2/134 - Train Accuracy: 0.3187, Validation Accuracy: 0.3525, Loss: 4.8495
Epoch   0 Batch    3/134 - Train Accuracy: 0.3180, Validation Accuracy: 0.3496, Loss: 4.0863
Epoch   0 Batch    4/134 - Train Accuracy: 0.3047, Validation Accuracy: 0.3434, Loss: 3.8765
Epoch   0 Batch    5/134 - Train Accuracy: 0.3312, Validation Accuracy: 0.3616, Loss: 3.5498
Epoch   0 Batch    6/134 - Train Accuracy: 0.3681, Validation Accuracy: 0.3704, Loss: 3.2742
Epoch   0 Batch    7/134 - Train Accuracy: 0.3433, Validation Accuracy: 0.3821, Loss: 3.3470
Epoch   0 Batch    8/134 - Train Accuracy: 0.3505, Validation Accuracy: 0.3869, Loss: 3.1870
Epoch   0 Batch    9/134 - Train Accuracy: 0.4023, Validation Accuracy: 0.4024, Loss: 2.9140
Epoch   0 Batch   10/134 - Train Accuracy: 0.3597, Validation Accuracy: 0.3885, Loss: 3.0788
Epoch   0 Batch   11/134 - Train Accuracy: 0.3804, Validation Accuracy

Epoch   0 Batch   90/134 - Train Accuracy: 0.4761, Validation Accuracy: 0.5042, Loss: 1.1705
Epoch   0 Batch   91/134 - Train Accuracy: 0.5565, Validation Accuracy: 0.5199, Loss: 1.0156
Epoch   0 Batch   92/134 - Train Accuracy: 0.5002, Validation Accuracy: 0.5251, Loss: 1.1315
Epoch   0 Batch   93/134 - Train Accuracy: 0.5012, Validation Accuracy: 0.5199, Loss: 1.1242
Epoch   0 Batch   94/134 - Train Accuracy: 0.4987, Validation Accuracy: 0.5229, Loss: 1.1147
Epoch   0 Batch   95/134 - Train Accuracy: 0.5099, Validation Accuracy: 0.5276, Loss: 1.1195
Epoch   0 Batch   96/134 - Train Accuracy: 0.5005, Validation Accuracy: 0.5211, Loss: 1.1115
Epoch   0 Batch   97/134 - Train Accuracy: 0.5032, Validation Accuracy: 0.5233, Loss: 1.0992
Epoch   0 Batch   98/134 - Train Accuracy: 0.4798, Validation Accuracy: 0.5289, Loss: 1.1567
Epoch   0 Batch   99/134 - Train Accuracy: 0.5119, Validation Accuracy: 0.5311, Loss: 1.0697
Epoch   0 Batch  100/134 - Train Accuracy: 0.5073, Validation Accuracy

Epoch   1 Batch   47/134 - Train Accuracy: 0.5791, Validation Accuracy: 0.5981, Loss: 0.7166
Epoch   1 Batch   48/134 - Train Accuracy: 0.5767, Validation Accuracy: 0.5966, Loss: 0.7120
Epoch   1 Batch   49/134 - Train Accuracy: 0.5950, Validation Accuracy: 0.5941, Loss: 0.6998
Epoch   1 Batch   50/134 - Train Accuracy: 0.5861, Validation Accuracy: 0.6028, Loss: 0.6936
Epoch   1 Batch   51/134 - Train Accuracy: 0.5802, Validation Accuracy: 0.5972, Loss: 0.6923
Epoch   1 Batch   52/134 - Train Accuracy: 0.5705, Validation Accuracy: 0.5874, Loss: 0.6854
Epoch   1 Batch   53/134 - Train Accuracy: 0.5693, Validation Accuracy: 0.5832, Loss: 0.6931
Epoch   1 Batch   54/134 - Train Accuracy: 0.5621, Validation Accuracy: 0.5737, Loss: 0.6735
Epoch   1 Batch   55/134 - Train Accuracy: 0.5920, Validation Accuracy: 0.5859, Loss: 0.6568
Epoch   1 Batch   56/134 - Train Accuracy: 0.5808, Validation Accuracy: 0.5992, Loss: 0.6689
Epoch   1 Batch   57/134 - Train Accuracy: 0.5965, Validation Accuracy

Epoch   2 Batch    4/134 - Train Accuracy: 0.6353, Validation Accuracy: 0.6490, Loss: 0.5347
Epoch   2 Batch    5/134 - Train Accuracy: 0.6300, Validation Accuracy: 0.6431, Loss: 0.5402
Epoch   2 Batch    6/134 - Train Accuracy: 0.6548, Validation Accuracy: 0.6414, Loss: 0.5020
Epoch   2 Batch    7/134 - Train Accuracy: 0.6254, Validation Accuracy: 0.6318, Loss: 0.5228
Epoch   2 Batch    8/134 - Train Accuracy: 0.6328, Validation Accuracy: 0.6326, Loss: 0.5190
Epoch   2 Batch    9/134 - Train Accuracy: 0.6633, Validation Accuracy: 0.6409, Loss: 0.4911
Epoch   2 Batch   10/134 - Train Accuracy: 0.6467, Validation Accuracy: 0.6468, Loss: 0.5201
Epoch   2 Batch   11/134 - Train Accuracy: 0.6460, Validation Accuracy: 0.6495, Loss: 0.5161
Epoch   2 Batch   12/134 - Train Accuracy: 0.6518, Validation Accuracy: 0.6411, Loss: 0.4937
Epoch   2 Batch   13/134 - Train Accuracy: 0.6096, Validation Accuracy: 0.6201, Loss: 0.5238
Epoch   2 Batch   14/134 - Train Accuracy: 0.6206, Validation Accuracy

Epoch   2 Batch   93/134 - Train Accuracy: 0.6774, Validation Accuracy: 0.6731, Loss: 0.4093
Epoch   2 Batch   94/134 - Train Accuracy: 0.6893, Validation Accuracy: 0.6837, Loss: 0.4111
Epoch   2 Batch   95/134 - Train Accuracy: 0.6968, Validation Accuracy: 0.6925, Loss: 0.4042
Epoch   2 Batch   96/134 - Train Accuracy: 0.6851, Validation Accuracy: 0.6893, Loss: 0.4083
Epoch   2 Batch   97/134 - Train Accuracy: 0.6623, Validation Accuracy: 0.6806, Loss: 0.4119
Epoch   2 Batch   98/134 - Train Accuracy: 0.6314, Validation Accuracy: 0.6694, Loss: 0.4370
Epoch   2 Batch   99/134 - Train Accuracy: 0.6817, Validation Accuracy: 0.6922, Loss: 0.4032
Epoch   2 Batch  100/134 - Train Accuracy: 0.6803, Validation Accuracy: 0.6866, Loss: 0.4091
Epoch   2 Batch  101/134 - Train Accuracy: 0.6714, Validation Accuracy: 0.6852, Loss: 0.4136
Epoch   2 Batch  102/134 - Train Accuracy: 0.6697, Validation Accuracy: 0.6797, Loss: 0.4183
Epoch   2 Batch  103/134 - Train Accuracy: 0.6920, Validation Accuracy

...

Epoch  97 Batch   13/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9853, Loss: 0.0009
Epoch  97 Batch   14/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9851, Loss: 0.0008
Epoch  97 Batch   15/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9856, Loss: 0.0008
Epoch  97 Batch   16/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9857, Loss: 0.0004
Epoch  97 Batch   17/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9847, Loss: 0.0008
Epoch  97 Batch   18/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9837, Loss: 0.0010
Epoch  97 Batch   19/134 - Train Accuracy: 0.9994, Validation Accuracy: 0.9830, Loss: 0.0014
Epoch  97 Batch   20/134 - Train Accuracy: 0.9994, Validation Accuracy: 0.9842, Loss: 0.0013
Epoch  97 Batch   21/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9864, Loss: 0.0011
Epoch  97 Batch   22/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9870, Loss: 0.0006
Epoch  97 Batch   23/134 - Train Accuracy: 0.9998, Validation Accuracy

Epoch  97 Batch  102/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9871, Loss: 0.0010
Epoch  97 Batch  103/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9869, Loss: 0.0008
Epoch  97 Batch  104/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9871, Loss: 0.0008
Epoch  97 Batch  105/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9867, Loss: 0.0004
Epoch  97 Batch  106/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9868, Loss: 0.0008
Epoch  97 Batch  107/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9874, Loss: 0.0008
Epoch  97 Batch  108/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9884, Loss: 0.0007
Epoch  97 Batch  109/134 - Train Accuracy: 0.9998, Validation Accuracy: 0.9869, Loss: 0.0014
Epoch  97 Batch  110/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9867, Loss: 0.0006
Epoch  97 Batch  111/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9856, Loss: 0.0004
Epoch  97 Batch  112/134 - Train Accuracy: 1.0000, Validation Accuracy

Epoch  98 Batch   59/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9849, Loss: 0.0006
Epoch  98 Batch   60/134 - Train Accuracy: 0.9988, Validation Accuracy: 0.9847, Loss: 0.0012
Epoch  98 Batch   61/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9853, Loss: 0.0005
Epoch  98 Batch   62/134 - Train Accuracy: 0.9994, Validation Accuracy: 0.9849, Loss: 0.0006
Epoch  98 Batch   63/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9849, Loss: 0.0004
Epoch  98 Batch   64/134 - Train Accuracy: 0.9995, Validation Accuracy: 0.9847, Loss: 0.0008
Epoch  98 Batch   65/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9850, Loss: 0.0008
Epoch  98 Batch   66/134 - Train Accuracy: 0.9990, Validation Accuracy: 0.9832, Loss: 0.0008
Epoch  98 Batch   67/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9841, Loss: 0.0010
Epoch  98 Batch   68/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9835, Loss: 0.0004
Epoch  98 Batch   69/134 - Train Accuracy: 1.0000, Validation Accuracy

Epoch  99 Batch   16/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9844, Loss: 0.0003
Epoch  99 Batch   17/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9852, Loss: 0.0008
Epoch  99 Batch   18/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9859, Loss: 0.0005
Epoch  99 Batch   19/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9858, Loss: 0.0004
Epoch  99 Batch   20/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9860, Loss: 0.0005
Epoch  99 Batch   21/134 - Train Accuracy: 0.9999, Validation Accuracy: 0.9863, Loss: 0.0006
Epoch  99 Batch   22/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9857, Loss: 0.0008
Epoch  99 Batch   23/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9864, Loss: 0.0004
Epoch  99 Batch   24/134 - Train Accuracy: 0.9992, Validation Accuracy: 0.9870, Loss: 0.0005
Epoch  99 Batch   25/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9866, Loss: 0.0005
Epoch  99 Batch   26/134 - Train Accuracy: 1.0000, Validation Accuracy

Epoch  99 Batch  105/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9850, Loss: 0.0009
Epoch  99 Batch  106/134 - Train Accuracy: 0.9985, Validation Accuracy: 0.9841, Loss: 0.0019
Epoch  99 Batch  107/134 - Train Accuracy: 0.9997, Validation Accuracy: 0.9834, Loss: 0.0009
Epoch  99 Batch  108/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9835, Loss: 0.0011
Epoch  99 Batch  109/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9832, Loss: 0.0013
Epoch  99 Batch  110/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9838, Loss: 0.0014
Epoch  99 Batch  111/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9832, Loss: 0.0006
Epoch  99 Batch  112/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9822, Loss: 0.0006
Epoch  99 Batch  113/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9816, Loss: 0.0006
Epoch  99 Batch  114/134 - Train Accuracy: 1.0000, Validation Accuracy: 0.9813, Loss: 0.0007
Epoch  99 Batch  115/134 - Train Accuracy: 0.9994, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [2]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [3]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence = sentence.lower()
    words = sentence.split(' ')
    ids = []
    for w in words:
        if(w in vocab_to_int):
            ids.append(vocab_to_int[w])
        else:
            ids.append(vocab_to_int['<UNK>'])
  
    return ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [6]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [155, 117, 70, 17, 71, 37, 25]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [353, 196, 265, 186, 53, 266, 109, 279, 1]
  French Words: il a vu un vieux camion jaune . <EOS>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.